In [56]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical

from PIL import Image

In [57]:
df = pd.read_csv('/kaggle/input/train-with-paths/train.csv')

In [ ]:
category = 'Women Tshirts'

In [ ]:
df = df[df['Category'] == category]

In [59]:
# For Kaggle

df['image_path'] = df.apply(lambda x: '/kaggle/input/visual-taxonomy/train_images/' + x['image_path'].split('/')[-1], axis=1)

In [60]:
df.head()


,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,attr_6,attr_7,attr_8,attr_9,attr_10,image_path
0,0,Men Tshirts,5,default,round,printed,default,short sleeves,NaN,NaN,NaN,NaN,NaN,/kaggle/input/visual-taxonomy/train_images/000...
1,1,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN,/kaggle/input/visual-taxonomy/train_images/000...
2,2,Men Tshirts,5,default,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN,/kaggle/input/visual-taxonomy/train_images/000...
3,3,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN,/kaggle/input/visual-taxonomy/train_images/000...
4,4,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,NaN,NaN,NaN,NaN,NaN,/kaggle/input/visual-taxonomy/train_images/000...


In [61]:
df.dropna(axis=1, how='all' , inplace=True)

In [62]:
df.head()

,id,Category,len,attr_1,attr_2,attr_3,attr_4,attr_5,image_path
0,0,Men Tshirts,5,default,round,printed,default,short sleeves,/kaggle/input/visual-taxonomy/train_images/000...
1,1,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,/kaggle/input/visual-taxonomy/train_images/000...
2,2,Men Tshirts,5,default,polo,solid,solid,short sleeves,/kaggle/input/visual-taxonomy/train_images/000...
3,3,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,/kaggle/input/visual-taxonomy/train_images/000...
4,4,Men Tshirts,5,multicolor,polo,solid,solid,short sleeves,/kaggle/input/visual-taxonomy/train_images/000...


In [63]:
df.isna().sum()

id               0
Category         0
len              0
attr_1        1257
attr_2        1123
attr_3        1476
attr_4        1318
attr_5        1290
image_path       0
dtype: int64

In [ ]:
labelcolumns = []
for i in df.columns:
    if i.startswith('attr'): labelcolumns.append(i)

In [65]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomBrightness(0.1),
])

In [66]:
def prepare_images(image_paths, image_size=(224, 224)):
    """
    Converts image paths into augmented tensors using predefined data augmentation layers.
    
    Args:
        image_paths (list of str): List of image paths.
        image_size (tuple): Target size of images (height, width).
    
    Returns:
        np.array: Array of preprocessed and augmented image tensors.
    """

    images = []
    
    for path in image_paths:
        try:
            img = Image.open(path).convert("RGB")
            img = img.resize(image_size)  
            img_array = np.array(img) / 255.0  
            
            # Add batch dimension and convert to tensor
            img_tensor = tf.convert_to_tensor(img_array, dtype=tf.float32)
            img_tensor = tf.expand_dims(img_tensor, axis=0)  
            
            # Apply data augmentation
            augmented_img = data_augmentation(img_tensor)[0].numpy()  
            images.append(augmented_img)

        except Exception as e:
            print(f"Error loading image {path}: {e}")
    
    return np.array(images)


In [67]:
def predict_labels(unique_labels, x_train, y_train, x_predict):
    """
    Train a ResNet50-based classifier and predict labels.
    
    Args:
        unique_labels (int): Number of unique labels in the dataset.
        x_train (np.array): Preprocessed training images.
        y_train (np.array): Labels for training images.
        x_predict (np.array): Preprocessed images to predict.
    
    Returns:
        list of dict: List of dictionaries with structure {'id': <id>, 'label': <label>}.
    """
    # Convert labels to categorical
    y_train_categorical = to_categorical(y_train, num_classes=unique_labels)
    
    # Load ResNet50 with pretrained weights
    base_model = ResNet50(weights='/kaggle/input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze the base model
    
    # Build a simple classification model
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(256, activation='relu'),
        Dense(unique_labels, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model
    model.fit(x_train, y_train_categorical, epochs=4, batch_size=32, verbose=1)
    
    # Predict labels for x_predict
    predictions = model.predict(x_predict)
    y_predict = np.argmax(predictions, axis=1)
    
    return y_predict


In [ ]:
for i in labelcolumns:
    predict_df = df[df[i].isna()][['id' , 'image_path']]
    train_df = df[~df[i].isna()][['id' , 'image_path' , i]]
    unique_val = len(train_df[i].unique())

    le = LabelEncoder()

    train_df[i] = le.fit_transform(train_df[i])

    class_to_label = dict(zip(le.classes_, range(len(le.classes_))))
    label_to_class = {v:k  for k, v in class_to_label.items()}

    y_train = train_df[i]

    x_train = prepare_images(train_df['image_path'])
    x_predict = prepare_images(predict_df['image_path'])

    # Step 2: Predict labels
    y_predict = predict_labels(
        unique_val,
        x_train,
        y_train,
        x_predict
    )
    
    # Step 3: Prepare final result
    predict_df['label'] = [label_to_class[label] for label in y_predict]
    result = predict_df[['id', 'label']]

    result.to_csv(f'/kaggle/working/{category}_{i}.csv', index=False)


Epoch 1/4
188/188 ━━━━━━━━━━━━━━━━━━━━ 26s 99ms/step - accuracy: 0.3364 - loss: 1.3621
Epoch 2/4
188/188 ━━━━━━━━━━━━━━━━━━━━ 18s 94ms/step - accuracy: 0.3899 - loss: 1.2948
Epoch 3/4
188/188 ━━━━━━━━━━━━━━━━━━━━ 19s 101ms/step - accuracy: 0.3845 - loss: 1.2791
Epoch 4/4
188/188 ━━━━━━━━━━━━━━━━━━━━ 19s 100ms/step - accuracy: 0.3972 - loss: 1.2658
40/40 ━━━━━━━━━━━━━━━━━━━━ 9s 156ms/step
Epoch 1/4
192/192 ━━━━━━━━━━━━━━━━━━━━ 25s 90ms/step - accuracy: 0.5866 - loss: 0.6946
Epoch 2/4
192/192 ━━━━━━━━━━━━━━━━━━━━ 18s 95ms/step - accuracy: 0.6990 - loss: 0.5279
Epoch 3/4
192/192 ━━━━━━━━━━━━━━━━━━━━ 19s 101ms/step - accuracy: 0.7095 - loss: 0.4955
Epoch 4/4
192/192 ━━━━━━━━━━━━━━━━━━━━ 19s 98ms/step - accuracy: 0.7022 - loss: 0.4818
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 162ms/step
Epoch 1/4
181/181 ━━━━━━━━━━━━━━━━━━━━ 26s 99ms/step - accuracy: 0.5808 - loss: 0.6921
Epoch 2/4
181/181 ━━━━━━━━━━━━━━━━━━━━ 17s 94ms/step - accuracy: 0.6887 - loss: 0.5443
Epoch 3/4
181/181 ━━━━━━━━━━━━━━━━━━━━ 18s 10

In [69]:
print('done')

done
